In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import csv
import scipy
from scipy.optimize import curve_fit

In [3]:
ase1_k = 32.25
ase1_l0 = 2.28
kinesin_k = 32.25
kinesin_l0 = 3.12
ot_k = 20

In [4]:
def sumsqrdiff(avgv, v):
    s = []
    for i in range(0,min(len(avgv),len(v))):
        s.append((v[i]-avgv[i])**2)
    return s

In [5]:
def elsum(vec1, vec2):
    s = []
    for i in range(0,min(len(vec1),len(vec2))):
        s.append(vec1[i]+vec2[i])
    return s

In [6]:
def distance(vec1, vec2):
    return np.sqrt((vec1[0]-vec2[0])**2+(vec1[1]-vec2[1])**2+(vec1[2]-vec2[2])**2)

In [7]:
def calc_force_ot(file, kot):
    with open(file,'r') as file:
        csv_reader = csv.reader(file, delimiter=' ')
        delta = 0
        process_mts = False
        process_header = False
        time = []
        n_anchors = 0
        mts = []
        force = []
        ri = [0,0,0]
        rj = [0,0,0]
        for row in csv_reader:
            if row[0] == "n_steps":
                process_header = True
            elif row[0] == "time":
                time.append(float(row[2]))
            elif row[0] == "n_members":
                mts.append(float(row[2]))
                force.append(0.0)

            elif row[0] == "position[0]":
                process_mts=True
            # Data rows
            elif process_header:
                delta = row[2]
                process_header = False
            elif process_mts:
                force[-1]=force[-1]-kot*(50-float(row[1]))
                process_mts=False
    return time, force

In [8]:
def calc_force_xlinks(file, k, L0):
    with open(file,'r') as file:
        csv_reader = csv.reader(file, delimiter=' ')
        delta = 0
        process_xlinks = False
        process_header = False
        process_anchors = False
        n_doubly = []
        time = []
        n_anchors = 0
        xlinks = []
        doubly = False
        force = []
        ri = [0,0,0]
        rj = [0,0,0]
        for row in csv_reader:
            if row[0] == "n_steps":
                process_header = True
            elif row[0] == "time":
                time.append(float(row[2]))
            elif row[0] == "n_members":
                xlinks.append(float(row[2]))
            elif row[0] == "is_doubly":
                process_xlinks = True
                n_doubly.append(0)
                force.append(0)
            elif row[0] == "bound":
                process_xlinks = False
                process_anchors = True
                n_anchors = 0

            # Data rows
            elif process_header:
                delta = row[2]
                process_header = False
            elif process_xlinks:
                if row[0] == "1":
                    doubly = True
                    n_doubly[-1] = n_doubly[-1] + 1
                else:
                    doubly = False
            elif process_anchors:
                if n_anchors == 1: #two anchors per xlink            
                    if row[10] == 3:
                        ri = [float(row[3]), float(row[4]), float(row[5])]
                    else:
                        rj = [float(row[3]), float(row[4]), float(row[5])]
                    if doubly == True and distance(ri, rj) > 0:
                        force[-1] = force[-1] + k*(1.0-L0/distance(ri, rj))*(ri[1]-rj[1])
                    process_anchors = False
                    process_xlinks = True
                else:
                    if row[10] == 3:
                        ri = [float(row[3]), float(row[4]), float(row[5])]
                    else:
                        rj = [float(row[3]), float(row[4]), float(row[5])]
                    n_anchors = n_anchors + 1
            else:
                print("Error- unknown row")
    return time, force


In [9]:
def analyze_output_forces(file_format, nran, nvar):
    forces = [[]]*nvar
    sigmas = [[]]*nvar
    times = [[]]*nvar
    ns = [0]*nvar
    for ran_index in range (0,nran):
        for var_index in range (0,nvar):
            file = eval(f"f'{file_format}'")
            time, force = calc_force_ot(file, ot_k)
            time.pop()
            force.pop()
            if len(force)>2:
                if (len(forces[var_index]) == 0):
                    forces[var_index] = force
                    times[var_index] = time
                    ns[var_index] = 1
                else:
                    forces[var_index] = elsum(forces[var_index], force)
                    times[var_index] = elsum(times[var_index], time)
                    ns[var_index] = ns[var_index]+1

    for var_index in range (0,nvar):
        forces[var_index] = np.divide(forces[var_index], ns[var_index])
        times[var_index] = np.divide(times[var_index], ns[var_index])

    for ran_index in range (0,nran):
        for var_index in range (0,nvar):
            file = eval(f"f'{file_format}'")
            time, force = calc_force_ot(file, ot_k)
            time.pop();
            force.pop();
            if len(force)>2:
                if (len(sigmas[var_index]) == 0):
                    sigmas[var_index] = [0]*len(forces[var_index])
                sigmas[var_index] = elsum(sigmas[var_index], sumsqrdiff(forces[var_index], force));

    for var_index in range (0,nvar):
        sigmas[var_index] = np.divide((sigmas[var_index]), (ns[var_index]-1)*ns[var_index])
        sigmas[var_index] = np.sqrt(sigmas[var_index])
    return times, forces, sigmas

In [10]:
#nvar = 5
#nran = 15
#file_format = "./ratioscanThreadsX2/example{var_index}/ot_test_v{var_index:03d}_r{ran_index:03d}_rigid_filament_microtubuleSpec.txt"
#times, forces, sigmas = analyze_output_forces(file_format, nran, nvar)
#for j in range (0,nvar):
#    plt.errorbar(times[j],forces[j],sigmas[j])
#plt.xlabel("Time (sim units)")
#plt.ylabel("Force (sim units)")
#plt.legend(["Xlink concentration at 1 nM","2 nM","3 nM","4 nM","5 nM"])

In [11]:
def fit(t,A,tau):
    return np.multiply(A,(np.subtract(1.0,np.exp(np.subtract(0.0,np.divide(t,tau))))))

In [12]:
#taus = [];
#dtaus = [];
#As = [];
#dAs = [];
#for j in range (0,5):
#    p,pcov=curve_fit(fit,times[j],forces[j])
#    perr=np.sqrt(np.diag(pcov))
#    taus.append(p[1]);
#    dtaus.append(perr[1]);
#    As.append(p[0]);
#    dAs.append(perr[0]);